In [14]:
%load_ext pycodestyle_magic

In [8]:
#  count_food1.py

from collections import Counter

import spacy
import pandas as pd

# 宮島の口コミのファイルの読み込み
df = pd.read_csv('text/miyajima_sample.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

# 口コミに含まれている空行を削除
df['口コミ'] = df['口コミ'].replace('\n+', '\n', regex=True)

nlp = spacy.load("ja_ginza")

# ルールの追加
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [
    {"label": "Dish", "pattern": [
        {"TEXT": "あなご"},
        {"TEXT": {"REGEX": r"飯|めし"}},
    ]},
    {"label": "Dish", "pattern": "揚げもみじ"},
    {"label": "Dish", "pattern": "ガンス"},
    {"label": "Dish", "pattern": "がんす"},
    {"label": "Dish", "pattern": [
        {"TEXT": "にぎり"},
        {"TEXT": "天"}
    ]}
]
ruler.add_patterns(patterns)

In [9]:
# 食べ物に関する単語をリストに追加
food = []
for doc in nlp.pipe(df["口コミ"]):
    # 食べ物に関する固有表現を抽出
    for ent in doc.ents:
        if ent.label_ == "Dish" or ent.label_ == "Food_other":
            food.append(ent.text)

# 単語の頻度をカウント
counter = Counter(food)

print('count word')
for word, count in counter.most_common():
    print(f'{count:>5} {word}')

count word
   35 もみじ饅頭
   27 揚げもみじ
    9 焼き牡蠣
    6 あなごめし
    5 にぎり天
    5 あなご飯
    5 ガンス
    5 コーヒー
    5 穴子
    4 牡蠣
    4 アイスクリーム
    4 お茶
    3 穴子飯
    3 牡蠣屋
    3 おにぎり
    3 コッペパン
    2 牡蠣料理
    2 牡蠣屋定食
    2 スタバ
    2 宮島ビール
    2 せんべい
    2 穴子めし
    2 チーズ
    2 ビール
    2 日本酒
    2 鹿せんべい
    1 カキフライ
    1 牡蠣飯
    1 カクテル
    1 ジュース
    1 白湯
    1 竹輪
    1 がんす
    1 ステーキ
    1 ケーキ
    1 秋ごろ
    1 アイス
    1 饅頭
    1 だし
    1 穴子入り竹輪
    1 宮島名物
    1 揚げ紅葉
    1 抹茶ミルクティー
    1 クリーム
    1 お菓子
    1 お好み焼き
    1 ごはん
    1 揚げ紅葉うまい
    1 揚げ紅葉饅頭
    1 パスタ
    1 海鮮丼
    1 牡蠣入り
    1 カレーパン
    1 焼きたてのもみじ饅頭
    1 焼き立て
    1 スターバックスコーヒー
    1 ねぎたこ
    1 レモン汁
    1 醤油マヨ
    1 牡蠣小屋
    1 桐葉菓
    1 焼きがきのはやし


In [16]:
#  count_food2.py

# 表記ゆれを解消するための用語辞書
# キーは代表語，値は表記ゆれの単語のリスト
name2variants = {"あなご飯": ["穴子飯", "あなごめし", "穴子めし"],
                 "揚げもみじ": ["揚げ紅葉", "揚げもみじ饅頭"],
                 "ガンス": ["がんす"]}

# 用語辞書を，表記揺れをチェックしやすい形式に変換
# キーは表記ゆれの単語，値は代表語
variant2name = {}
for name, variants in name2variants.items():
    variant2name[name] = name    
    for variant in variants:
        variant2name[variant] = name

# 表記ゆれを解消して単語の頻度をカウント
counter2 = {}    # 表記ゆれを解消した後の単語の頻度を保存するための辞書
for word, count in counter.most_common():
    if word in variant2name:
        if variant2name[word] in counter2:
            counter2[variant2name[word]] += count
        else:
            counter2[variant2name[word]] = count
    else:
        counter2[word] = count

print('count word')
for word, count in sorted(counter2.items(), key=lambda x: -x[1]):
    print(f'{count:>5} {word}')

count word
   35 もみじ饅頭
   28 揚げもみじ
   16 あなご飯
    9 焼き牡蠣
    6 ガンス
    5 にぎり天
    5 コーヒー
    5 穴子
    4 牡蠣
    4 アイスクリーム
    4 お茶
    3 牡蠣屋
    3 おにぎり
    3 コッペパン
    2 牡蠣料理
    2 牡蠣屋定食
    2 スタバ
    2 宮島ビール
    2 せんべい
    2 チーズ
    2 ビール
    2 日本酒
    2 鹿せんべい
    1 カキフライ
    1 牡蠣飯
    1 カクテル
    1 ジュース
    1 白湯
    1 竹輪
    1 ステーキ
    1 ケーキ
    1 秋ごろ
    1 アイス
    1 饅頭
    1 だし
    1 穴子入り竹輪
    1 宮島名物
    1 抹茶ミルクティー
    1 クリーム
    1 お菓子
    1 お好み焼き
    1 ごはん
    1 揚げ紅葉うまい
    1 揚げ紅葉饅頭
    1 パスタ
    1 海鮮丼
    1 牡蠣入り
    1 カレーパン
    1 焼きたてのもみじ饅頭
    1 焼き立て
    1 スターバックスコーヒー
    1 ねぎたこ
    1 レモン汁
    1 醤油マヨ
    1 牡蠣小屋
    1 桐葉菓
    1 焼きがきのはやし


In [22]:
#  count_food_man.py

from collections import Counter

import spacy
import pandas as pd

# 宮島の口コミのファイルの読み込み
df = pd.read_csv('text/miyajima_sample.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

# 口コミに含まれている空行を削除
df['口コミ'] = df['口コミ'].replace('\n+', '\n', regex=True)

df_man = df[df['性別']=='男性']['口コミ'] 

nlp = spacy.load("ja_ginza")

# ルールの追加
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [
    {"label": "Dish", "pattern": [
        {"TEXT": "あなご"},
        {"TEXT": {"REGEX": r"飯|めし"}},
    ]},
    {"label": "Dish", "pattern": "揚げもみじ"},
    {"label": "Dish", "pattern": "ガンス"},
    {"label": "Dish", "pattern": "がんす"},
    {"label": "Dish", "pattern": [
        {"TEXT": "にぎり"},
        {"TEXT": "天"}
    ]}
]
ruler.add_patterns(patterns)

# 食べ物に関する単語をリストに追加
food = []
for doc in nlp.pipe(df_man):
    # 食べ物に関する固有表現を抽出
    for ent in doc.ents:
        if ent.label_ == "Dish" or ent.label_ == "Food_other":
            food.append(ent.text)

# 単語の頻度をカウント
counter = Counter(food)

#  count_food2.py

# 表記ゆれを解消するための用語辞書
# キーは代表語，値は表記ゆれの単語のリスト
name2variants = {"あなご飯": ["穴子飯", "あなごめし", "穴子めし"],
                 "揚げもみじ": ["揚げ紅葉", "揚げもみじ饅頭"],
                 "ガンス": ["がんす"]}

# 用語辞書を，表記揺れをチェックしやすい形式に変換
# キーは表記ゆれの単語，値は代表語
variant2name = {}
for name, variants in name2variants.items():
    variant2name[name] = name    
    for variant in variants:
        variant2name[variant] = name

# 表記ゆれを解消して単語の頻度をカウント
counter2 = {}    # 表記ゆれを解消した後の単語の頻度を保存するための辞書
for word, count in counter.most_common():
    if word in variant2name:
        if variant2name[word] in counter2:
            counter2[variant2name[word]] += count
        else:
            counter2[variant2name[word]] = count
    else:
        counter2[word] = count

print('count word')
for word, count in sorted(counter2.items(), key=lambda x: -x[1]):
    print(f'{count:>5} {word}')


count word
   24 もみじ饅頭
   13 揚げもみじ
   10 あなご飯
    6 焼き牡蠣
    5 穴子
    3 牡蠣
    3 お茶
    2 コッペパン
    2 スタバ
    2 せんべい
    2 チーズ
    2 日本酒
    2 鹿せんべい
    1 白湯
    1 コーヒー
    1 ステーキ
    1 ケーキ
    1 饅頭
    1 だし
    1 穴子入り竹輪
    1 クリーム
    1 お菓子
    1 お好み焼き
    1 ごはん
    1 揚げ紅葉うまい
    1 海鮮丼
    1 にぎり天
    1 牡蠣料理
    1 レモン汁
    1 ビール
    1 宮島ビール
    1 醤油マヨ
    1 牡蠣小屋
    1 桐葉菓
    1 焼きがきのはやし
    1 牡蠣屋


In [23]:
#  count_food_woman.py

from collections import Counter

import spacy
import pandas as pd

# 宮島の口コミのファイルの読み込み
df = pd.read_csv('text/miyajima_sample.csv', encoding='utf-8',
                 parse_dates=['旅行の時期'])

# 口コミに含まれている空行を削除
df['口コミ'] = df['口コミ'].replace('\n+', '\n', regex=True)

df_woman = df[df['性別']=='女性']['口コミ'] 

nlp = spacy.load("ja_ginza")

# ルールの追加
ruler = nlp.add_pipe("entity_ruler", before="ner")
patterns = [
    {"label": "Dish", "pattern": [
        {"TEXT": "あなご"},
        {"TEXT": {"REGEX": r"飯|めし"}},
    ]},
    {"label": "Dish", "pattern": "揚げもみじ"},
    {"label": "Dish", "pattern": "ガンス"},
    {"label": "Dish", "pattern": "がんす"},
    {"label": "Dish", "pattern": [
        {"TEXT": "にぎり"},
        {"TEXT": "天"}
    ]}
]
ruler.add_patterns(patterns)

# 食べ物に関する単語をリストに追加
food = []
for doc in nlp.pipe(df_man):
    # 食べ物に関する固有表現を抽出
    for ent in doc.ents:
        if ent.label_ == "Dish" or ent.label_ == "Food_other":
            food.append(ent.text)

# 単語の頻度をカウント
counter = Counter(food)

#  count_food2.py

# 表記ゆれを解消するための用語辞書
# キーは代表語，値は表記ゆれの単語のリスト
name2variants = {"あなご飯": ["穴子飯", "あなごめし", "穴子めし"],
                 "揚げもみじ": ["揚げ紅葉", "揚げもみじ饅頭"],
                 "ガンス": ["がんす"]}

# 用語辞書を，表記揺れをチェックしやすい形式に変換
# キーは表記ゆれの単語，値は代表語
variant2name = {}
for name, variants in name2variants.items():
    variant2name[name] = name    
    for variant in variants:
        variant2name[variant] = name

# 表記ゆれを解消して単語の頻度をカウント
counter2 = {}    # 表記ゆれを解消した後の単語の頻度を保存するための辞書
for word, count in counter.most_common():
    if word in variant2name:
        if variant2name[word] in counter2:
            counter2[variant2name[word]] += count
        else:
            counter2[variant2name[word]] = count
    else:
        counter2[word] = count

print('count word')
for word, count in sorted(counter2.items(), key=lambda x: -x[1]):
    print(f'{count:>5} {word}')


count word
   15 揚げもみじ
   11 もみじ饅頭
    6 ガンス
    6 あなご飯
    4 にぎり天
    4 アイスクリーム
    4 コーヒー
    3 焼き牡蠣
    3 おにぎり
    2 牡蠣屋
    2 牡蠣屋定食
    1 牡蠣料理
    1 カキフライ
    1 牡蠣飯
    1 牡蠣
    1 カクテル
    1 ジュース
    1 コッペパン
    1 竹輪
    1 宮島ビール
    1 秋ごろ
    1 アイス
    1 宮島名物
    1 抹茶ミルクティー
    1 揚げ紅葉饅頭
    1 ビール
    1 パスタ
    1 牡蠣入り
    1 カレーパン
    1 焼きたてのもみじ饅頭
    1 お茶
    1 焼き立て
    1 スターバックスコーヒー
    1 ねぎたこ
